# Term sparsity

**Adapted from**: Example 3.5 of [WML20b]

[WML20a] Wang, Jie, Victor Magron, and Jean-Bernard Lasserre.
*TSSOS: A Moment-SOS hierarchy that exploits term sparsity*.
arXiv preprint arXiv:1912.08899 (2020).

[WML20b] Wang, Jie, Victor Magron, and Jean-Bernard Lasserre.
*Chordal-TSSOS: a moment-SOS hierarchy that exploits term sparsity with chordal extension*.
arXiv preprint arXiv:2003.03210 (2020).

In [1]:
using DynamicPolynomials
@polyvar x[1:3]

(DynamicPolynomials.PolyVar{true}[x₁, x₂, x₃],)

We would like to find the minimum value of the polynomial

In [2]:
poly = x[1]^2 - 2x[1]*x[2] + 3x[2]^2 - 2x[1]^2*x[2] + 2x[1]^2*x[2]^2 - 2x[2]*x[3] + 6x[3]^2 + 18x[2]^2*x[3] - 54x[2]*x[3]^2 + 142x[2]^2*x[3]^2

2x₁²x₂² + 142x₂²x₃² - 2x₁²x₂ + 18x₂²x₃ - 54x₂x₃² + x₁² - 2x₁x₂ + 3x₂² - 2x₂x₃ + 6x₃²

The minimum value of the polynomial can be found to be zero.

In [3]:
import CSDP
solver = CSDP.Optimizer
using SumOfSquares
function sos_min(sparsity)
    model = Model(solver)
    @variable(model, t)
    @objective(model, Max, t)
    con_ref = @constraint(model, poly - t in SOSCone(), sparsity = sparsity)
    optimize!(model)
    return value(t), moment_matrix(con_ref)
end

bound, ν = sos_min(NoSparsity())
bound

Iter:  7 Ap: 9.00e-01 Pobj:  0.0000000e+00 Ad: 1.00e+00 Dobj:  5.8619776e-14 
Success: SDP solved
Primal objective value: 0.0000000e+00 
Dual objective value: 0.0000000e+00 
Relative primal infeasibility: 1.75e-09 
Relative dual infeasibility: 2.83e-50 
Real Relative Gap: 0.00e+00 
XZ Relative Gap: 8.00e-50 
DIMACS error measures: 2.63e-09 0.00e+00 2.83e-50 0.00e+00 0.00e+00 8.00e-50
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.71e-01 Pobj: -2.4909930e+02 Ad: 8.28e-01 Dobj:  7.2430886e+02 
Iter:  2 Ap: 9.99e-01 Pobj: -1.6816440e+02 Ad: 9.32e-01 Dobj:  1.6018610e+02 
Iter:  3 Ap: 9.96e-01 Pobj: -6.7498086e+00 Ad: 9.24e-01 Dobj:  6.1399335e+01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.4722865e+00 Ad: 9.01e-01 Dobj:  8.9328561e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -5.0478289e-01 Ad: 7.90e-01 Dobj:  2.7911669e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -1.5409183e-01 Ad: 7.76e-01 Dobj:  8.7430956e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -4.8559289e-02 Ad: 8.12e-01 

-3.0451607901937905e-10

We find the corresponding minimizer `(0, 0, 0)` by matching the moments
of the moment matrix with a dirac measure centered at this minimizer.

In [4]:
extractatoms(ν, 1e-6)

Atomic measure on the variables x[1], x[2], x[3] with 1 atoms:
 at [2.070354244219943e-6, 1.2367885264633775e-6, 2.2360576846083365e-8] with weight 1.000000001490363

We can see below that the basis contained 6 monomials hence we needed to use 6x6 PSD matrix variables.

In [5]:
ν.basis

MonomialBasis{DynamicPolynomials.Monomial{true}, DynamicPolynomials.MonomialVector{true}}(DynamicPolynomials.Monomial{true}[x₁x₂, x₂x₃, x₁, x₂, x₃, 1])

Using the monomial/term sparsity method of [WML20a] based on cluster completion, we find the same bound.

In [6]:
bound, ν = sos_min(MonomialSparsity())
bound

Iter: 14 Ap: 9.68e-01 Pobj: -3.0451608e-10 Ad: 9.70e-01 Dobj: -2.0275412e-07 
Success: SDP solved
Primal objective value: -3.0451608e-10 
Dual objective value: -2.0275412e-07 
Relative primal infeasibility: 2.24e-13 
Relative dual infeasibility: 1.55e-09 
Real Relative Gap: -2.02e-07 
XZ Relative Gap: 2.86e-09 
DIMACS error measures: 2.42e-13 0.00e+00 2.56e-09 0.00e+00 -2.02e-07 2.86e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.71e-01 Pobj: -2.4909930e+02 Ad: 8.28e-01 Dobj:  7.2430886e+02 
Iter:  2 Ap: 9.99e-01 Pobj: -1.6816440e+02 Ad: 9.32e-01 Dobj:  1.6018610e+02 
Iter:  3 Ap: 9.96e-01 Pobj: -6.7498086e+00 Ad: 9.24e-01 Dobj:  6.1399335e+01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.4722865e+00 Ad: 9.01e-01 Dobj:  8.9328561e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -5.0478289e-01 Ad: 7.90e-01 Dobj:  2.7911669e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -1.5409183e-01 Ad: 7.76e-01 Dobj:  8.7430956e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -4.8559289e-02 Ad: 8.12e

-3.0451607901937905e-10

Which is not suprising as no sparsity reduction could be performed.

In [7]:
[sub.basis for sub in ν.sub_moment_matrices]

1-element Vector{MonomialBasis{DynamicPolynomials.Monomial{true}, DynamicPolynomials.MonomialVector{true}}}:
 MonomialBasis{DynamicPolynomials.Monomial{true}, DynamicPolynomials.MonomialVector{true}}(DynamicPolynomials.Monomial{true}[x₁x₂, x₂x₃, x₁, x₂, x₃, 1])

Using the monomial/term sparsity method of [WML20b] based on chordal completion, the lower bound is smaller than 0.

In [8]:
bound, ν = sos_min(MonomialSparsity(ChordalCompletion()))
bound

Iter: 14 Ap: 9.68e-01 Pobj: -3.0451608e-10 Ad: 9.70e-01 Dobj: -2.0275412e-07 
Success: SDP solved
Primal objective value: -3.0451608e-10 
Dual objective value: -2.0275412e-07 
Relative primal infeasibility: 2.24e-13 
Relative dual infeasibility: 1.55e-09 
Real Relative Gap: -2.02e-07 
XZ Relative Gap: 2.86e-09 
DIMACS error measures: 2.42e-13 0.00e+00 2.56e-09 0.00e+00 -2.02e-07 2.86e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.17e-01 Pobj: -3.5364812e+02 Ad: 6.71e-01 Dobj:  5.8532180e+02 
Iter:  2 Ap: 8.26e-01 Pobj: -3.6133495e+02 Ad: 9.36e-01 Dobj:  1.9736716e+02 
Iter:  3 Ap: 9.70e-01 Pobj: -1.1800069e+02 Ad: 8.23e-01 Dobj:  1.4747690e+02 
Iter:  4 Ap: 1.00e+00 Pobj: -1.2898462e+01 Ad: 8.32e-01 Dobj:  5.5286769e+01 
Iter:  5 Ap: 1.00e+00 Pobj: -2.4900158e+00 Ad: 9.07e-01 Dobj:  8.7767866e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -1.2196157e+00 Ad: 8.79e-01 Dobj:  2.3717906e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -4.3509342e-01 Ad: 6.82e

-0.0035512009905827036

However, this bound was obtained with an SDP with 4 matrices of size 3x3.

In [9]:
[sub.basis for sub in ν.sub_moment_matrices]

4-element Vector{MonomialBasis{DynamicPolynomials.Monomial{true}, DynamicPolynomials.MonomialVector{true}}}:
 MonomialBasis{DynamicPolynomials.Monomial{true}, DynamicPolynomials.MonomialVector{true}}(DynamicPolynomials.Monomial{true}[x₂x₃, x₂, x₃])
 MonomialBasis{DynamicPolynomials.Monomial{true}, DynamicPolynomials.MonomialVector{true}}(DynamicPolynomials.Monomial{true}[x₂x₃, x₂, 1])
 MonomialBasis{DynamicPolynomials.Monomial{true}, DynamicPolynomials.MonomialVector{true}}(DynamicPolynomials.Monomial{true}[x₁, x₂, 1])
 MonomialBasis{DynamicPolynomials.Monomial{true}, DynamicPolynomials.MonomialVector{true}}(DynamicPolynomials.Monomial{true}[x₁x₂, x₁, 1])

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*